In [ ]:
!pip install nltk Sastrawi swifter

In [ ]:
import pandas as pd
import string
import numpy as np
import re
import nltk
from nltk.corpus import stopwords

In [ ]:
# Unduh korpus stopwords
nltk.download('stopwords')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Membuat variabel untuk dataset
nama_file = '/content/dataset.csv'

# Membaca dataset
dataset = pd.read_csv(nama_file)

In [ ]:
dataset.head()

In [ ]:
dataset.shape

In [ ]:
data = dataset['text']

Case Folding & cleaning text

In [ ]:
def clean_text(text):
    # Pastikan nilai merupakan string
    if isinstance(text, str):
        #hapus nama
        text = re.sub('@[^\s]+', '', text)
        #hapus tanda baca
        text = text.translate(str.maketrans('', '', string.punctuation))
        #ganti karakter html dengan tanda petik
        text = re.sub('<,*?>', '', text)
        #pertimbangkan huruf dengan angka
        text = re.sub('[^a-zA-Z]', ' ', text)
        #ganti line baru dengan spasi
        text = re.sub("\n", " ", text)
        #ubah ke huruf kecil
        text = text.lower()
        #hapus single char
        text = re.sub(r'\b[a-zA-Z]\b', ' ', text)
        #pisah dan gabungkan kata
        text = ' '.join(text.split())
        return text
    else:
        return ""

In [ ]:
dataset['clean'] = [clean_text(i) for i in data]
data = dataset['clean']

In [ ]:
dataset.head(10)

In [ ]:
print(dataset)

Tokenizing

In [ ]:
from nltk.tokenize import RegexpTokenizer
regexp = RegexpTokenizer(r'\w+|$[0-9]+|\S+')
dataset['Token'] = dataset['clean'].apply(regexp.tokenize)

In [ ]:
dataset

Normalisasi

In [ ]:
normalized_word = pd.read_csv("normalisasi.csv", encoding='latin1')

normalized_word_dict = {}
for index, row in normalized_word.iterrows():
    if row[0] not in normalized_word_dict:
        normalized_word_dict[row[0]] = row[1]

def normalized_tern(document):
    return [normalized_word_dict[tern] if tern in normalized_word_dict else tern for tern in document]

dataset['normalisasi'] = dataset['Token'].apply(normalized_tern)
dataset

hapus kata tidak perlu / remove stopword

In [ ]:
!dir

dataset.csv  normalisasi.csv  sample_data  stopwords.txt


In [ ]:
from nltk.corpus import stopwords
stopword = stopwords.words('indonesian')
txt_stopword = pd.read_csv("stopwords.txt", names=["stopwords"], header = None)

stopword.extend(["klo", "kl", "yg", "dgn", "dg", "g", "klrg", "dmn", "utk", "tdk", "sy",
                   "gk", "jg", "nya", "bgt", "blh", "sih", "udh", "deh", "tp",
                   "klo", "dll", "tpi", "krg", "enk", "dngn", "blg"])

stopword.extend(txt_stopword["stopwords"][0].split(' '))
stopword = set(stopword)

def stopwords(text):
  text = [word for word in text if word not in stopword]
  return text
dataset['Stopwords'] = dataset['normalisasi'].apply(stopwords)
dataset

Stemming / membakukan kata

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter

factory=StemmerFactory()
stemmer = factory.create_stemmer()

term_dict = {}

data = dataset['Stopwords']
for doc in data:
  for term in doc:
    if term not in term_dict:
      term_dict[term]=' '
  print(len(term_dict))
  print("...............")

for tern in term_dict:
  term_dict[term] = stemmer.stem(term)
  print(term,":",term_dict[term])

  print(len(term_dict))
  print("..................")

In [ ]:
def get_stemmed_term(doc):
  return [term_dict[term] for term in doc]

dataset['Stemmer'] = dataset['Stopwords'].apply(get_stemmed_term)
dataset

In [ ]:
data = pd.DataFrame(dataset)
data.to_excel ('teks_processing.xlsx', index = False, header=True)